
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>


<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.3 - alpha preview</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

# Getting Underway - Publishing assets
In this notebook, we will explore how to publish an Asset using Ocean Protocol. An Asset consists of several files
which are kept private, and optionally other links which are open (samples, descriptions, etc.).

A publisher will require access to two services;
1. A service to store the MetaData of the asset (part of the DDO) - 'Aquarius'
1. A service to manage permissioned access to the assets - 'Brizo'

The publishing of an asset consists of;
1. Preparing the asset files locally
1. Preparing the metadata of the asset
1. Uploading assets or otherwise making them available as URL's
1. Registering the metadata and service endpoints into Aquarius
1. Registering the asset into the Blockchain (into the DID Registry)

<p><img src="https://raw.githubusercontent.com/oceanprotocol/mantaray/develop/doc/img/jupyter_cell.png" alt="drawing" width="400" align="center"/></p>
<p><b>Overall client and service architecture</b></p>

### Section 0: Import modules, connect the Ocean Protocol API

In [1]:
# Standard imports
import logging

# Import mantaray and the Ocean API (squid)
import random
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils
from mantaray_utilities.user import password_map
from pprint import pprint
# Setup logging
manta_utils.logging.logger.setLevel('CRITICAL')
from time import sleep

10 - logging         - <module>        - Logging started


Using default logging settings.


In [2]:
# Get the configuration file path for this environment
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))
logging.critical("Squid API version: {}".format(squid_py.__version__))

50 - <ipython-input-2-84136f362abd> - <module>        - Deployment type: DEFAULT
50 - <ipython-input-2-84136f362abd> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_local.ini
50 - <ipython-input-2-84136f362abd> - <module>        - Squid API version: 0.4.2


In [3]:
# Instantiate Ocean with the default configuration file.
configuration = Config(CONFIG_INI_PATH)
ocn = Ocean(configuration)


### Section 1: A publisher account in Ocean

In [4]:
# List the accounts in the network
print(len(ocn.accounts.list()), "accounts exist")

# Print a simple table listing accounts and balances
print("{:<5} {:<45} {:<20} {}".format("","Address", "Ocean Token Balance","ETH balance"))
for i, acct in enumerate(ocn.accounts.list()):
    acct_balance = ocn.accounts.balance(acct)
    print("{:<5} {:<45} {:<20} {}".format(i,acct.address, acct_balance.ocn, acct_balance.eth))

3 accounts exist
      Address                                       Ocean Token Balance  ETH balance
0     0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e    4992                 1000000001878471200000000000
1     0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0    1100                 1000000000000000000000000000
2     0xA99D43d86A0758d5632313b8fA3972B6088A21BB    2                    1000000000000000000000000000


In [5]:
# Get a publisher account
path_passwords = manta_utils.config.get_project_path() / 'passwords.csv'
passwords = manta_utils.user.load_passwords(path_passwords)

publisher_acct = random.choice([acct for acct in ocn.accounts.list() if password_map(acct.address, passwords)])
publisher_acct.password = password_map(publisher_acct.address, passwords)
assert publisher_acct.password

In [6]:
print("Publisher account address: {}".format(publisher_acct.address))
print("Publisher account balance:", ocn.accounts.balance(publisher_acct).ocn)

Publisher account address: 0xA99D43d86A0758d5632313b8fA3972B6088A21BB
Publisher account balance: 2


Your account will need some Ocean Token to make real transactions, let's ensure that you are funded!

In [7]:
# ensure Ocean token balance
if ocn.accounts.balance(publisher_acct).ocn == 0:
    ocn.accounts.request_tokens(publisher_acct, 100)

### Section 2: Create the Metadata for your asset
The metadata is a key-value set of attributes which describe your asset

A more complex use case is to manually generate your metadata conforming to Ocean standard, but for demonstration purposes,
a utility in squid-py is used to generate a sample Meta Data dictionary.

In [9]:
# Get a simple example of Meta Data from the library directly
metadata = squid_py.ddo.metadata.Metadata.get_example()
print('Name of asset:', metadata['base']['name'])
# Print the entire (JSON) dictionary
pprint(metadata)

Name of asset: Ocean protocol white paper
{'additionalInformation': {'structuredMarkup': [{'mediaType': 'application/ld+json',
                                                 'uri': 'http://skos.um.es/unescothes/C01194/jsonld'},
                                                {'mediaType': 'text/turtle',
                                                 'uri': 'http://skos.um.es/unescothes/C01194/turtle'}],
                           'updateFrequency': 'yearly'},
 'base': {'author': 'Ocean Protocol Foundation Ltd.',
          'categories': ['white-papers'],
          'compression': '',
          'contentType': 'text/csv',
          'copyrightHolder': 'Ocean Protocol Foundation Ltd.',
          'dateCreated': '2012-10-10T17:00:000Z',
          'description': 'Introduce the main concepts and vision behind ocean '
                         'protocol',
          'encoding': 'UTF-8',
          'files': [{'checksum': 'efb2c764274b745f5fc37f97c6b0e761',
                     'checksumType': 'MD

Note that the price is included in the Metadata! This will be purchase price you are placing on the asset. You can
Alter the metadata object at any time before publishing.

In [10]:
print("Price of Asset:", metadata['base']['price'])
metadata['base']['price'] = 9
print("Updated price of Asset:", metadata['base']['price'])

Price of Asset: 23
Updated price of Asset: 9


Let's inspect another important component of your metadata - the actual asset files. The files of an asset are
described by valid URL's. You are responsible for ensuring the URL's are alive. Files may have additional
information, including a checksum, length, content type, etc.

In [12]:
for i, file in enumerate(metadata['base']['files']):
    print("Asset link {}: {}".format( i, file['url']))

Asset link 0: https://testocnfiles.blob.core.windows.net/testfiles/testzkp.pdf
Asset link 1: s3://ocean-test-osmosis-data-plugin-dataseeding-1537375953/data.txt
Asset link 2: http://www3.cs.stonybrook.edu/~algorith/implement/graphbase/distrib/cweb3.4e.tar.gz


## Section 3 Publish the asset
With this metadata object prepared, we are ready to publish the asset into Ocean Protocol.

The result will be an ID string (DID) registered into the smart contract, and a DID Document stored in Aquarius.
The asset URLS's are encrypted upon publishing.

In [13]:
ddo = ocn.assets.create(metadata, publisher_acct)
registered_did = ddo.did
print("New asset registered at", registered_did)

New asset registered at did:op:abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78


Inspect the new DDO. We can retrieve the DDO as a dictionary object, feel free to explore the DDO in the cell below!

In [14]:
ddo_dict = ddo.as_dictionary()
print("DID:", ddo.did)
print("Services within this DDO:")
for svc in ddo_dict['service']:
    print(svc['type'], svc['serviceEndpoint'])

DID: did:op:abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78
Services within this DDO:
Access http://172.15.0.17:8030/api/v1/brizo/services/consume
Authorization http://localhost:12001
Metadata http://172.15.0.15:5000/api/v1/aquarius/assets/ddo/did:op:abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78


Note that the 'files' attribute has been replaced by the 'encryptedFiles' attribute!

In [15]:
assert 'files' not in ddo.metadata['base']
print("Encryped 'files' attribute, everything safe and secure!")
print("Encrypted files decrypt on purchase! [{}...] etc. ".format(ddo.metadata['base']['encryptedFiles'][:50]))

Encryped 'files' attribute, everything safe and secure!
Encrypted files decrypt on purchase! [0xdd26bdaf8397d32f2ee513a973cb7ae2243c2eedb8f059ae...] etc. 


## Section 4: Verify your asset
Now, let's verify that this asset exists in the MetaData storage.

A call to assets.resolve() will call the Aquarius service and retrieve the DID Document

In [16]:
ddo = ocn.assets.resolve(registered_did)
print("Asset '{}' resolved from Aquarius metadata storage: {}".format(ddo.did,ddo.metadata['base']['name']))

Asset 'did:op:abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78' resolved from Aquarius metadata storage: Ocean protocol white paper


For illustrative purposes, this is the error you can expect if the DID is *NOT* in the database

In [17]:
random_did = 'did:op:9a3c2693c1f942b8a61cba7d212e5cd50c1b9a5299f74e39848e9b4c2148d453'
try: ocn.assets.resolve(random_did)
except Exception as e: print("(This raises, as required)", e)

(This raises, as required) DID "0x9a3c2693c1f942b8a61cba7d212e5cd50c1b9a5299f74e39848e9b4c2148d453" is not found on-chain in the current did registry. Please ensure assets are registered in the correct keeper contracts. The keeper-contracts DIDRegistry address is 0x611f28Ef25D778aFC5a0034Aea94297e2c215a42


Similarly, we can verify that this asset is registered into the blockchain, and that you are the owner.

Congratulations on publishing an Asset into Ocean Protocol!

In [18]:
# We need the pure ID string as in the DID registry (a DID without the prefixes)
asset_id = squid_py.did.did_to_id(registered_did)
owner = ocn._keeper.did_registry.contract_concise.getOwner(asset_id)
print("Asset ID", asset_id, "owned by", owner)
assert str.lower(owner) == str.lower(publisher_acct.address)

Asset ID abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78 owned by 0xA99D43d86A0758d5632313b8fA3972B6088A21BB


Next, let's search for our assets in Ocean Protocol